In [89]:
import sys
sys.path.append('../RQ2/')
import bootstrap_ssr

In [90]:
def check_batch_bisect():
    bootstrap_ssr.batch_total = 0
    bootstrap_ssr.batch_durations = 0
    
    #test case 1: All passing builds
    
    durations = [2,2,2,2,2,2,2,2] #not using durations anymore
    grouped_batch = [1,1,1,1,1,1,1,1] #not using grouped_batches anymore
    actual_group_results = [1,1,1,1,1,1,1,1]
    
    bootstrap_ssr.batch_bisect(grouped_batch, actual_group_results, durations)
    
    assert bootstrap_ssr.batch_total == 1, "Should be 1"
    
    #test case 2: All failing builds
    bootstrap_ssr.batch_total = 0
    
    actual_group_results = [0,0,0,0,0,0,0,0]
    bootstrap_ssr.batch_bisect(grouped_batch, actual_group_results, durations)
    assert bootstrap_ssr.batch_total == 15, "Should be 15"
    
    #test case 3: odd batch sizes
    bootstrap_ssr.batch_total = 0
    
    actual_group_results = [1,0,1,1,1,1,1,1,1]
    bootstrap_ssr.batch_bisect(grouped_batch, actual_group_results, durations)
    assert bootstrap_ssr.batch_total == 7, "Should be 7"
    

In [91]:
def check_batch_stop_4():
    bootstrap_ssr.batch_total = 0
    bootstrap_ssr.batch_durations = 0
    
    #test case 1: All passing builds
    
    durations = [2,2,2,2,2,2,2,2] #not using durations anymore
    grouped_batch = [1,1,1,1,1,1,1,1] #not using grouped_batches anymore
    actual_group_results = [1,1,1,1,1,1,1,1]
    
    bootstrap_ssr.batch_stop_4(grouped_batch, actual_group_results, durations)
    
    assert bootstrap_ssr.batch_total == 1, "Should be 1"
    
    #test case 2: All failing builds
    bootstrap_ssr.batch_total = 0
    
    actual_group_results = [0,0,0,0,0,0,0,0]
    bootstrap_ssr.batch_stop_4(grouped_batch, actual_group_results, durations)
    assert bootstrap_ssr.batch_total == 11, "Should be 15"
    
    #test case 3: Batch size != 4x
    bootstrap_ssr.batch_total = 0
    
    actual_group_results = [1,0,1,1,1,1,1,1,1,1]
    bootstrap_ssr.batch_stop_4(grouped_batch, actual_group_results, durations)
    assert bootstrap_ssr.batch_total == 9, "Should be 9"

In [87]:
#Since most of the data is obtained through code, remainder of test cases will check the validity of the results
#There is a file 'New_Updated_RQ_Results.csv' which records the builds required, prediction based delays,
#                             grouping based delays and so on, for all heuristics of all batch sizes and batching algorithms

In [106]:
results = pd.read_csv('../New_Updated_RQ_Results.csv')

In [108]:
for i in range(len(results)):
    row = results.iloc[i]
    b = row['builds_reqd']
    d = row['batch_delays']
    
    
    
    if row['method'] == 'ssr':
        #if there is a build, there is a batch => there are batching delays
        if b > 0:
            if d == 0:
                if row['batch_size'] > 1:  #batch size of 1 has no delays
                    assert False, "Should incur batching delay"

        #if there are no builds, no batches are made => no batching delays are incurred
        if b == 0:
            if d != 0:
                assert False, "Batching delay should not occur"
    
    
    
    
    if row['method'] == 'ml':
        #if there is a build, there is a batch => there are batching delays
        if b > 0:
            if d == 0:
                if row['batch_size'] > 1:  #batch size of 1 has no delays
                    assert False, "Should incur batching delay"

        #if there are no builds, no batches are made => no batching delays are incurred
        if b == 0:
            if d != 0:
                assert False, "Batching delay should not occur"
    
    
    
    if row['method'] == 'sbs':
        #If there are grouping based delays
        if d > 0:
            assert False, "SmartBuildSkip has no batching delays"
        
        #If there are no builds made in the project, then no failing builds are caught and prediction delay is incurred
        if b == 0:
            if row['sbs_delays'] == 0:
                assert False, "There should be a delay"
    
    
    
    
    if row['method'] == 'gr':
        #If there are prediction delays
        if row['sbs_delays'] > 0:
            assert False, "Batching has no prediction delays"
        
        #If there are no builds, then no batches are made and batching delay must be 0
        if b == 0:
            if d != 0:
                assert False, "There should not be a delay"
        
        #If builds are made, batches are made and batching delay is incurred
        if b > 0:
            if d == 0:
                assert False, "There should be a delay"